In [1]:
# -*- coding: utf-8 -*-
import dataiku
from dataiku import spark as dkuspark
import dataiku.spark as dspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import udf
import pandas as pd
import numpy as np
import requests
import ffmpeg
import speech_recognition as sr
import subprocess
from pydub import AudioSegment
import os
import scipy.io.wavfile as wav

sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

MAX_EPISODES = 500


r = sr.Recognizer()

episodes_sample = dataiku.Dataset("episodes_sample_filtered")

episodes_sample_df = dspark.get_dataframe(sqlContext, episodes_sample)

mp3_folder = dataiku.Folder("mp3_files_local")
wav_folder = dataiku.Folder("wav_files_local")


/data/dataiku/data_dir/code-envs/python/ffmpeg_27/local/lib/python2.7/dist-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:

def read_episode(url):

    import subprocess

    import requests
    from dlffmpeg import getFFmpeg
    dl = getFFmpeg()
    dl.path = ''
    dl.silent = True
    dl.pretty = True
    dl.verbose = False
    dl.run()
    
    #file = requests.get(url)
#    mp3_folder = dataiku.Folder("mp3_files_local")
    #with open("audio.mp3", 'wb') as w:
    #    w.write(file.content)

#    from pydub import AudioSegment
#    AudioSegment.from_mp3("audio.mp3").export("/tmp/audio.wav", format="wav")
#    mp3_folder.delete_path(mp3_folder.get_path() + "/" + audio_id + ".mp3")
    subprocess.call(["ffmpeg","-y",
                             "-i","audio.mp3",
                             "-r","16000",
                             "-ac","1",
                             "-segment_time","00:00:50",
                             "-f","segment",
                             "audio_part_%03d.wav"])
    return url

read_udf = udf(lambda z: read_episode(z), StringType())


In [3]:

rdf = episodes_sample_df
rdf2 = rdf.withColumn( 'url_out',read_udf('audio_url'))



# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a SparkSQL dataframe
#episodes_read_df = rdf2.to_pandas() # For this sample code, simply copy input to output

# Write recipe outputs
episodes_read = dataiku.Dataset("episodes_read")
dkuspark.write_with_schema(episodes_read, rdf2)

Py4JJavaError: An error occurred while calling o65.savePyDataFrame.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3, 172.31.15.192, executor 4): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 248, in <lambda>
    func = lambda _, it: map(mapper, it)
  File "<string>", line 1, in <lambda>
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 85, in <lambda>
    return lambda *a: f(*a)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-2-ee542a2c5f6

In [1]:
# -*- coding: utf-8 -*-
import dataiku
from dataiku import spark as dkuspark
import dataiku.spark as dspark

In [ ]:
dataiku.Folder